# San Francisco Rental Prices Dashboard

In this notebook, I will compile the visualizations from the previous analysis into functions that can be used for a Panel dashboard.

In [1]:
# Initial imports of libraries and their dependencies

import panel as pn
pn.extension('plotly')
import plotly.express as px
import pandas as pd
import hvplot.pandas
import matplotlib.pyplot as plt
import os
from pathlib import Path
from dotenv import load_dotenv

import warnings
warnings.filterwarnings('ignore')

AttributeError: module 'pandas' has no attribute 'Panel'

In [ ]:
# Read the Mapbox API key
load_dotenv()
map_box_api = os.getenv("mapbox")
px.set_mapbox_access_token(map_box_api)

# Print type of map_box_api to check if the class is 'str'

print(type(map_box_api))

# Import Data

In [ ]:
# Import the necessary CSVs to Pandas DataFrames

file_path = Path("Data/sfo_neighborhoods_census_data.csv")
location_path = Path("Data/neighborhoods_coordinates.csv")

sfo_data = pd.read_csv(file_path, index_col="year")
sf_location_data = pd.read_csv(location_path)

- - -

## Panel Visualizations

In this section, you will copy the code for each plot type from your analysis notebook and place it into separate functions that Panel can use to create panes for the dashboard. 

These functions will convert the plot object to a Panel pane.

Be sure to include any DataFrame transformation/manipulation code required along with the plotting code.

Return a Panel pane object from each function that can be used to build the dashboard.

Note: Remove any `.show()` lines from the code. We want to return the plots instead of showing them. The Panel dashboard will then display the plots.

In [ ]:
# Define Panel Visualization Functions
# Embedded plot codes in different functions for call back in separate group for Panel set up 
# used code from rental_analysis file - refer back for details

def housing_units_per_year():
    """Housing Units Per Year."""
    
    housing_units_grp = (sfo_data['housing_units'].groupby(['year']).mean())
    sfo_housing_units_df = pd.DataFrame(housing_units_grp).reset_index()
    housing_units_std = sfo_housing_units_df.housing_units.std()
    housing_units_min_1 = min(sfo_housing_units_df.housing_units)
    housing_units_max_1 = max(sfo_housing_units_df.housing_units)

    sfo_housing_units_df_plot = sfo_housing_units_df.hvplot.bar(x = 'year', y = 'housing_units',
                                                                ylim=(housing_units_min_1 - housing_units_std, housing_units_max_1 + housing_units_std), 
                                                                xlabel = 'Year', ylabel = 'Housing Units',
                                                                title = "Housing Units in San Francisco from 2010 to 2016",
                                                                yformatter="%.0f"
                                                                )
    
    return sfo_housing_units_df_plot


def average_gross_rent():
    """Average Gross Rent in San Francisco Per Year."""
    
    avg_rent_df = (sfo_data[['gross_rent']].groupby(['year']).mean()).reset_index()
    rent_std = avg_rent_df.gross_rent.std()
    rent_min = min(avg_rent_df.gross_rent) - rent_std
    rent_max = max(avg_rent_df.gross_rent) + rent_std
    average_gross_rent_plot = avg_rent_df.hvplot.line(x='year', y='gross_rent',title= "Average Gross Rent by Year", color = 'red',
                                                      ylim = (rent_min, rent_max), xlabel = "Year", ylabel= "Gross Rent")
    return average_gross_rent_plot

def average_sales_price():
    """Average Sales Price Per Year."""
    
    sqr_foot_df = (sfo_data[['sale_price_sqr_foot']].groupby(['year']).mean()).reset_index()
    sqr_std = sqr_foot_df.sale_price_sqr_foot.std()
    sqr_min = min(sqr_foot_df.sale_price_sqr_foot) - sqr_std
    sqr_max = max(sqr_foot_df.sale_price_sqr_foot) + sqr_std
    average_sqr_foot_plot = sqr_foot_df.hvplot.line(x='year', y='sale_price_sqr_foot',title= "Average Sale Price Per SqFt by Year", 
                        color = 'purple', ylim = (sqr_min, sqr_max), xlabel = "Year", ylabel= "Sale Price per SqrFt") 
    return average_sqr_foot_plot

def average_price_by_neighborhood():
    """Average Prices by Neighborhood."""
    
    neighborhood_grp = (sfo_data[['neighborhood', 'sale_price_sqr_foot', 'housing_units', 'gross_rent']]
                         .groupby(['year', 'neighborhood'])).mean()
    neighborhood_grp_df = pd.DataFrame(neighborhood_grp).reset_index()
    neighborhood_grp_df.rename(columns={'neighborhood': 'Neighborhood'}, inplace = True)
    df_line = neighborhood_grp_df.hvplot.line(x= 'year',
                                              y='sale_price_sqr_foot',
                                              xlabel= 'Year',
                                              ylabel= 'Average Sale Price per SqrFt',
                                              groupby= 'Neighborhood', 
                                              title = 'San Francisco Average Sales Price by Square Foot'
                                              )
    df_rent = neighborhood_grp_df.hvplot.line(x= 'year',
                                              y='gross_rent',
                                              xlabel= 'Year',
                                              ylabel= 'Average Gross Rent',
                                              groupby= 'Neighborhood',
                                              title = 'San Francisco Average Rent price'
                                              )
    return df_line + df_rent

def top_most_expensive_neighborhoods():
    """Top 10 Most Expensive Neighborhoods."""
    
    neighborhood_max_df = (sfo_data[['neighborhood', 'sale_price_sqr_foot', 'housing_units','gross_rent']]
                           .groupby(['neighborhood'])).mean()
    neighborhood_max_grp_df = pd.DataFrame(neighborhood_max_df).reset_index()
    neighborhood_max_grp_df['sale_price_sqr_foot'] = round(neighborhood_max_grp_df['sale_price_sqr_foot'], 2)
    neighborhood_max_grp_df = neighborhood_max_grp_df.sort_values(['sale_price_sqr_foot'], ascending = False)
    neighborhood_max_grp_df = neighborhood_max_grp_df.head(10)
    
    expensive_neighborhood_plot = neighborhood_max_grp_df.hvplot.bar(
        x = 'neighborhood',
        y = 'sale_price_sqr_foot',
        xlabel = "Neighborhoods",
        ylabel = 'Average Sales price per SqrFt',
        title = 'Top 10 Most Expensive Neighborhoods in San Francisco',
        rot = 90,
        height = 400
    )
    
    return expensive_neighborhood_plot

def most_expensive_neighborhoods_rent_sales():
    """Comparison of Rent and Sales Prices of Most Expensive Neighborhoods."""   
    combined_sale_rental_average = (sfo_data[['neighborhood', 'sale_price_sqr_foot', 'housing_units', 'gross_rent']]
                                    .groupby(['year', 'neighborhood'])).mean()
    combined_sale_rental_average = pd.DataFrame(combined_sale_rental_average).reset_index()
    combined_sale_rental_average.drop(columns= ['housing_units'], inplace = True)
    combined_sale_rental_average.rename(columns= {
        'year': 'Year', 'neighborhood': 'Neighborhood', 'gross_rent': 'Gross Rent', 'sale_price_sqr_foot': 'Sale Price per SqrFt'}, inplace = True)
    combined_sale_rental_average.set_index('Year', inplace = True)
    
    combined_df_plot = combined_sale_rental_average.hvplot.bar(
        xlabel= 'Neighborhoods',
        ylabel= 'Dollar Amount',
        groupby= 'Neighborhood',
        rot=90,
        height = 400
        )

    return combined_df_plot
    
def parallel_coordinates():
    """Parallel Coordinates Plot."""
    
    cost_analysis_df = (sfo_data[['neighborhood', 'sale_price_sqr_foot', 'housing_units', 'gross_rent']]
                        .groupby(['neighborhood'])).mean().reset_index()
    cost_analysis_df = cost_analysis_df.sort_values(['sale_price_sqr_foot'], ascending = False)
    cost_analysis_df['sale_price_sqr_foot'] = round(cost_analysis_df['sale_price_sqr_foot'], 2)
    cost_analysis_df['gross_rent'] = round(cost_analysis_df['gross_rent'], 2)
    cost_analysis_df.rename(columns={
        'neighborhood': 'Neighborhood', 'sale_price_sqr_foot': 'Average Sale Price per SqrFt', 
        'housing_units': 'Average Housing Units', 'gross_rent': 'Average Gross Rent'}, inplace = True)
    cost_analysis_df = cost_analysis_df.head(10)
    
    parallel_coordinates_plot = px.parallel_coordinates(cost_analysis_df,
                                                        dimensions=['Average Sale Price per SqrFt', 'Average Housing Units', 'Average Gross Rent'],
                                                        color='Average Sale Price per SqrFt',
                                                        title= 'Parallel Coordinates Analysis of Most Expensive SF Neighborhoods')
    
    return parallel_coordinates_plot

def parallel_categories():
    """Parallel Categories Plot."""
    
    cost_analysis_df = (sfo_data[['neighborhood', 'sale_price_sqr_foot', 'housing_units', 'gross_rent']]
                        .groupby(['neighborhood'])).mean().reset_index()
    cost_analysis_df = cost_analysis_df.sort_values(['sale_price_sqr_foot'], ascending = False)
    cost_analysis_df['sale_price_sqr_foot'] = round(cost_analysis_df['sale_price_sqr_foot'], 2)
    cost_analysis_df['gross_rent'] = round(cost_analysis_df['gross_rent'], 2)
    cost_analysis_df.rename(columns={
        'neighborhood': 'Neighborhood', 'sale_price_sqr_foot': 'Average Sale Price per SqrFt', 
        'housing_units': 'Average Housing Units', 'gross_rent': 'Average Gross Rent'}, inplace = True)
    cost_analysis_df = cost_analysis_df.head(10)

    parallel_categories_plot = px.parallel_categories(cost_analysis_df,
                                                      dimensions=['Neighborhood', 'Average Sale Price per SqrFt', 'Average Housing Units', 'Average Gross Rent'],
                                                      color='Average Sale Price per SqrFt',
                                                      title= 'Parallel Categories Analysis of Most Expensive SF Neighborhoods')
    return parallel_categories_plot
    
def neighborhood_map():
    """Neighborhood Map."""
    
    sf_combined_mean_df = (sfo_data[['neighborhood', 'sale_price_sqr_foot', 'housing_units', 'gross_rent']]
                           .groupby(['neighborhood'])).mean().reset_index()
    neighborhood_combined_loca_df = pd.concat([sf_location_data, sf_combined_mean_df], axis = 'columns', join = 'inner')
    neighborhood_combined_loca_df.drop(columns=['neighborhood'], inplace = True)
    px.set_mapbox_access_token(map_box_api)

    sf_location_map = px.scatter_mapbox(
        neighborhood_combined_loca_df,
        lat = 'Lat',
        lon = 'Lon',
        size = 'sale_price_sqr_foot',
        hover_name = 'Neighborhood',
        color = 'gross_rent',
        color_continuous_scale=px.colors.cyclical.IceFire,
        mapbox_style='carto-positron',
        size_max=15,
        zoom=11,
        height = 600,
        width = 800,
        title = 'Average Sale Price Per Square Foot and Gross Rent in San Francisco'
        )

    
    return sf_location_map

def sunburst():
    """Sunburst Plot."""
    neighborhood_max_df = (sfo_data[['neighborhood', 'sale_price_sqr_foot', 'housing_units','gross_rent']]
                           .groupby(['neighborhood'])).mean()
    neighborhood_max_grp_df = pd.DataFrame(neighborhood_max_df).reset_index()
    neighborhood_max_grp_df['sale_price_sqr_foot'] = round(neighborhood_max_grp_df['sale_price_sqr_foot'], 2)
    neighborhood_max_grp_df = neighborhood_max_grp_df.sort_values(['sale_price_sqr_foot'], ascending = False)
    neighborhood_max_grp_df_top10 = neighborhood_max_grp_df.head(10)
    c_sunburst = sfo_data[sfo_data["neighborhood"].isin(neighborhood_max_grp_df_top10["neighborhood"])]
    c_sunburst.sort_values('sale_price_sqr_foot', ascending = False, inplace = True)
    c_sunburst.reset_index(inplace = True)
    c_sunburst.rename(columns={
        'year': 'Year', 'neighborhood': 'Neighborhood', 'sale_price_sqr_foot': 'Average Sale Price per SqrFt', 
        'housing_units': 'Average Housing Units', 'gross_rent': 'Average Gross Rent'}, inplace = True)
    
    fig_map = px.sunburst(
        c_sunburst,
        path = ['Year', 'Neighborhood'],
        color = 'Average Gross Rent',
        color_continuous_scale=px.colors.cyclical.IceFire,
        width = 750,
        height = 750,
        title= "Cost Analysis of Most Expensive Neighborhoods in San Francisco"
        )
    
    return fig_map

## Panel Dashboard

In this section, you will combine all of the plots into a single dashboard view using Panel. Be creative with your dashboard design!

In [ ]:
# set the plots for panel dashboard (How we would like to display charts)

row_of_line = pn.Row(housing_units_per_year(), average_gross_rent(), average_sales_price())
row_of_line_interactive = pn.Row(average_price_by_neighborhood())
row_of_bar = pn.Row(top_most_expensive_neighborhoods())
row_of_bar_interactive =  pn.Row(most_expensive_neighborhoods_rent_sales())
row_of_parallel = pn.Row(parallel_categories(), parallel_coordinates())
row_of_neighborhood_map = pn.Row(neighborhood_map())
row_of_sunburst = pn.Row(sunburst())


In [ ]:
# Create a Title for the Dashboard and set columns layers
plots_as_column = pn.Column(
    "# San Francisco Neighborhood Cost Analysis", row_of_line, row_of_line_interactive, row_of_bar,
    row_of_bar_interactive, row_of_parallel, row_of_neighborhood_map, row_of_sunburst
)


In [ ]:
# Create a tab layout for the dashboard

tabs = pn.Tabs(
    ("All Plots", plots_as_column),
    ("Housing Unit Plots", row_of_line),
    ("Interactive Housing Costs Plots", row_of_line_interactive),
    ("SF Most Expensive Neigbhorhoods", row_of_bar),
    ("Interactive Neigbhorhoods Comparison", row_of_bar_interactive),
    ("SF Statistical Analysis", row_of_parallel),
    ("SF Neighborhood Map", row_of_neighborhood_map),
    ("SF Top 60 Neighbhorhood Sunburst", row_of_sunburst)
)

## Serve the Panel Dashboard

In [ ]:
# serve the Panel Dashboard on localhost

tabs.show()

# Debugging

Note: Some of the Plotly express plots may not render in the notebook through the panel functions.

However, you can test each plot by uncommenting the following code

In [ ]:
# housing_units_per_year()

In [ ]:
# average_gross_rent()

In [ ]:
# average_sales_price()

In [ ]:
# average_price_by_neighborhood()

In [ ]:
# top_most_expensive_neighborhoods()

In [ ]:
# most_expensive_neighborhoods_rent_sales()

In [ ]:
# neighborhood_map().show()

In [ ]:
# parallel_categories()

In [ ]:
# parallel_coordinates()

In [ ]:
# sunburst().show()